# 使用随机森林 用一年的过去天气数据来预测我们城市明天的最高温度

In [ ]:
import pandas as pd
features = pd.read_csv('../part03-neural-network/csv-data/temps.csv')
features.head(5)

数据解释
 * year：2016年所有数据点
 * month：一年中的月份数
 * day：一年中的某一天的数字
 * week：星期几
 * temp_2：2天前的最高温度
 * temp_1：1天前的最高温度
 * average： 历史平均最高温度
 * actual：最高温度 真实值

In [ ]:
print('The shape of our features is:', features.shape)
features.describe()

## 数据预处理
1. 查看数据的维度，我们注意到只有348行，但其实2016年有366天。通过NOAA的数据，我注意到几个缺失的日子
2. week 是字符，可以使用热编码，转成数字型

In [ ]:
features = pd.get_dummies(features)
print('The shape of our features is:', features.shape) # 数据形状现在是349 x 18
features.iloc[:,5:].head(5)

## 将数据转换为数组

In [ ]:
import numpy as np
labels = np.array(features['actual']) # Labels are the values we want to predict
features= features.drop('actual', axis = 1) # Remove the labels from the features
feature_list = list(features.columns) # Saving feature names
features = np.array(features) # Convert to numpy array


## 将数据分成训练和测试集

In [ ]:
from sklearn.model_selection import train_test_split
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 42)
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

训练和评估之前，需要建立一个基线作为明智的衡量标准

In [ ]:
# The baseline predictions are the historical averages
baseline_preds = test_features[:, feature_list.index('average')]
# Baseline errors, and display average baseline error
baseline_errors = abs(baseline_preds - test_labels)
print('Average baseline error: ', round(np.mean(baseline_errors), 2))

## 使用Scikit-learn创建和训练模型

1. 从skicit-learn导入随机森林回归模型
2. 实例化模型，并在训练数据上拟合（scikit-learn的训练名称）模型
3. 再次设置随机状态以获得可重现的结果

In [ ]:
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
rf.fit(train_features, train_labels);

## 对测试集进行预测

In [ ]:
predictions = rf.predict(test_features)
# Calculate the absolute errors
errors = abs(predictions - test_labels)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')
# 平均预估值为3.83度。 这比基线平均提高了1度。 虽然这看起来似乎并不重要，但它比基线好近25％，现实生活中，基线可能代表公司数百万美元。

## 确定性能指标
为了正确看待我们的预测，我们可以使用从100％中减去的平均百分比误差来计算准确度。

In [ ]:
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / test_labels)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.') # Accuracy: 93.99 %.

## 解释模型和报告结果
这个模型如何得出价值？ 有两种方法可以深入了解随机森林：
  首先，我们可以查看森林中的单个树，
  其次，我们可以查看解释变量的特征重要性。

### 可视化单个决策树
Skicit-learn中随机森林实施中最酷的部分之一是： 可以实际检查森林中的任何树木。

我们将选择一棵树，并将整棵树保存为图像

In [ ]:
# Import tools needed for visualization
from sklearn.tree import export_graphviz
from IPython.display import Image
import pydotplus
# Pull out one tree from the forest
tree = rf.estimators_[5]

dot_data = tree.export_graphviz(clf, out_file=None,
                         feature_names=iris.feature_names,
                         class_names=iris.target_names,
                         filled=True, rounded=True,
                         special_characters=True)
graph = pydotplus.graph_from_dot_data(dot_data)
Image(graph.create_png())

In [ ]:
# 使用 pydot 可以
import pydot
# Export the image to a dot file
export_graphviz(tree, out_file = 'tree.dot', feature_names = feature_list, rounded = True, precision = 1)

# Use dot file to create a graph
(graph, ) = pydot.graph_from_dot_file('tree.dot')

# Write graph to a png file
graph.write_png('tree.png')
